# ध्यान केन्द्रित गर्ने प्रणाली र ट्रान्सफर्मरहरू

पुनरावर्ती नेटवर्कहरूको एक प्रमुख कमजोरी यो हो कि अनुक्रमका सबै शब्दहरूले नतिजामा समान प्रभाव पार्छन्। यसले नामित इकाई पहिचान (Named Entity Recognition) र मेसिन अनुवाद (Machine Translation) जस्ता अनुक्रम-देखि-अनुक्रम कार्यहरूको लागि मानक LSTM एन्कोडर-डिकोडर मोडेलहरूसँग उपयुक्त प्रदर्शन गर्न सक्दैन। वास्तविकतामा, इनपुट अनुक्रमका केही विशेष शब्दहरूले अन्य शब्दहरूको तुलनामा अनुक्रमिक नतिजाहरूमा बढी प्रभाव पार्छन्।

मेसिन अनुवाद जस्ता अनुक्रम-देखि-अनुक्रम मोडेललाई विचार गर्नुहोस्। यो दुई पुनरावर्ती नेटवर्कहरूद्वारा कार्यान्वयन गरिन्छ, जहाँ एउटा नेटवर्क (**एन्कोडर**) इनपुट अनुक्रमलाई लुकेको अवस्थामा सङ्कुचित गर्छ, र अर्को नेटवर्क (**डिकोडर**) यस लुकेको अवस्थालाई अनुवादित नतिजामा विस्तार गर्छ। यस दृष्टिकोणको समस्या यो हो कि नेटवर्कको अन्तिम अवस्थाले वाक्यको सुरुवात सम्झन गाह्रो हुन्छ, जसले गर्दा लामो वाक्यहरूमा मोडेलको गुणस्तर कमजोर हुन्छ।

**ध्यान केन्द्रित गर्ने प्रणालीहरू** (Attention Mechanisms) ले प्रत्येक इनपुट भेक्टरको सन्दर्भात्मक प्रभावलाई RNN को प्रत्येक आउटपुट भविष्यवाणीमा तौल दिने माध्यम प्रदान गर्छ। यसलाई कार्यान्वयन गर्ने तरिका भनेको इनपुट RNN का मध्यवर्ती अवस्थाहरू र आउटपुट RNN बीच छोटो मार्गहरू सिर्जना गर्नु हो। यसरी, आउटपुट प्रतीक $y_t$ उत्पन्न गर्दा, हामी सबै इनपुट लुकेका अवस्थाहरू $h_i$ लाई विभिन्न तौल गुणांक $\alpha_{t,i}$ का साथ विचार गर्नेछौं।

![एन्कोडर/डिकोडर मोडेलमा एडिटिभ ध्यान तह देखाउने छवि](../../../../../translated_images/ne/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) मा एडिटिभ ध्यान प्रणाली भएको एन्कोडर-डिकोडर मोडेल, [यो ब्लग पोस्ट](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) बाट उद्धृत]*

ध्यान म्याट्रिक्स $\{\alpha_{i,j}\}$ ले इनपुटका निश्चित शब्दहरूले आउटपुट अनुक्रमको कुनै शब्द उत्पन्न गर्न कत्तिको भूमिका खेल्छन् भन्ने प्रतिनिधित्व गर्दछ। तल यस्तो म्याट्रिक्सको उदाहरण दिइएको छ:

![RNNsearch-50 द्वारा फेला परेको नमूना संरेखण देखाउने छवि, Bahdanau - arviz.org बाट लिइएको](../../../../../translated_images/ne/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) बाट लिइएको चित्र (Fig.3)*

ध्यान केन्द्रित गर्ने प्रणालीहरू प्राकृतिक भाषा प्रशोधनमा हालको वा निकट भविष्यको उत्कृष्ट अवस्थाको लागि जिम्मेवार छन्। तर ध्यान थप्दा मोडेलका प्यारामिटरहरूको संख्या धेरै बढ्छ, जसले गर्दा RNN हरूसँग स्केलिङ समस्याहरू उत्पन्न हुन्छ। RNN हरूको स्केलिङको प्रमुख बाधा भनेको मोडेलहरूको पुनरावर्ती प्रकृतिले प्रशिक्षणलाई ब्याच र समानान्तर बनाउन चुनौतीपूर्ण बनाउँछ। RNN मा अनुक्रमका प्रत्येक तत्वलाई क्रमिक रूपमा प्रशोधन गर्नुपर्छ, जसले गर्दा यसलाई सजिलै समानान्तर बनाउन सकिँदैन।

ध्यान केन्द्रित गर्ने प्रणालीहरूको अपनत्व र यस बाधाले आज हामीले BERT देखि OpenGPT3 सम्म प्रयोग गर्ने उत्कृष्ट ट्रान्सफर्मर मोडेलहरूको सिर्जना गर्‍यो।

## ट्रान्सफर्मर मोडेलहरू

प्रत्येक अघिल्लो भविष्यवाणीको सन्दर्भलाई अर्को मूल्याङ्कन चरणमा अगाडि बढाउने सट्टा, **ट्रान्सफर्मर मोडेलहरू** ले **स्थानगत एन्कोडिङहरू** र ध्यान प्रणाली प्रयोग गरेर दिइएको इनपुटको सन्दर्भलाई प्रदान गरिएको पाठको झ्यालभित्र समेट्छ। तलको छविले कसरी स्थानगत एन्कोडिङहरूले ध्यान प्रणालीसँग मिलेर झ्यालभित्रको सन्दर्भ समेट्न सक्छ भन्ने देखाउँछ।

![ट्रान्सफर्मर मोडेलहरूमा मूल्याङ्कन कसरी गरिन्छ भन्ने देखाउने एनिमेटेड GIF।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

किनभने प्रत्येक इनपुट स्थानलाई स्वतन्त्र रूपमा प्रत्येक आउटपुट स्थानमा म्याप गरिन्छ, ट्रान्सफर्मरहरूले RNN हरूको तुलनामा राम्रो समानान्तर गर्न सक्छन्, जसले गर्दा धेरै ठूलो र अभिव्यक्तिपूर्ण भाषा मोडेलहरू सक्षम हुन्छन्। प्रत्येक ध्यान हेडले शब्दहरू बीचका विभिन्न सम्बन्धहरू सिक्न प्रयोग गर्न सकिन्छ, जसले प्राकृतिक भाषा प्रशोधनका कार्यहरू सुधार गर्छ।

**BERT** (Bidirectional Encoder Representations from Transformers) एक धेरै ठूलो बहु-तह ट्रान्सफर्मर नेटवर्क हो, जसमा *BERT-base* का लागि १२ तहहरू र *BERT-large* का लागि २४ तहहरू छन्। मोडेललाई पहिलो पटक ठूलो पाठ डाटाको कर्पस (WikiPedia + पुस्तकहरू) मा असुपरभाइज्ड प्रशिक्षण (वाक्यमा लुकेका शब्दहरूको भविष्यवाणी) प्रयोग गरेर पूर्व-प्रशिक्षण गरिन्छ। पूर्व-प्रशिक्षणको क्रममा मोडेलले महत्त्वपूर्ण स्तरको भाषा बुझाइ प्राप्त गर्छ, जसलाई अन्य डाटासेटहरूसँग फाइन ट्युनिङ गरेर उपयोग गर्न सकिन्छ। यस प्रक्रियालाई **ट्रान्सफर लर्निङ** भनिन्छ।

![http://jalammar.github.io/illustrated-bert/ बाट चित्र](../../../../../translated_images/ne/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT, DistilBERT, BigBird, OpenGPT3 जस्ता ट्रान्सफर्मर आर्किटेक्चरका धेरै भेरिएसनहरू छन्, जसलाई फाइन ट्युन गर्न सकिन्छ। [HuggingFace package](https://github.com/huggingface/) ले यी आर्किटेक्चरहरूलाई PyTorch का साथ प्रशिक्षण गर्नका लागि रिपोजिटरी प्रदान गर्दछ।

## पाठ वर्गीकरणका लागि BERT प्रयोग गर्दै

हेरौं, कसरी हामी पूर्व-प्रशिक्षित BERT मोडेललाई हाम्रो परम्परागत कार्य समाधान गर्न प्रयोग गर्न सक्छौं: अनुक्रम वर्गीकरण। हामी हाम्रो मूल AG News डाटासेट वर्गीकृत गर्नेछौं।

पहिले, HuggingFace लाइब्रेरी र हाम्रो डाटासेट लोड गरौं:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


हामीले प्रि-ट्रेन गरिएको BERT मोडेल प्रयोग गर्ने भएकाले, हामीलाई विशेष टोकनाइजर प्रयोग गर्न आवश्यक हुनेछ। पहिलो चरणमा, हामी प्रि-ट्रेन गरिएको BERT मोडेलसँग सम्बन्धित टोकनाइजर लोड गर्नेछौं।

HuggingFace लाइब्रेरीमा प्रि-ट्रेन गरिएको मोडेलहरूको संग्रह छ, जसलाई तपाईं `from_pretrained` फङ्सनहरूमा तिनीहरूको नामहरू तर्कको रूपमा उल्लेख गरेर सजिलै प्रयोग गर्न सक्नुहुन्छ। मोडेलका लागि आवश्यक सबै बाइनरी फाइलहरू स्वतः डाउनलोड हुनेछ।

तर, केही अवस्थामा तपाईंलाई आफ्नै मोडेलहरू लोड गर्न आवश्यक पर्न सक्छ। यस्तो अवस्थामा, तपाईंले टोकनाइजरका लागि आवश्यक प्यारामिटरहरू, मोडेल प्यारामिटरहरू भएको `config.json` फाइल, बाइनरी वेटहरू लगायतका सबै सम्बन्धित फाइलहरू समावेश भएको डाइरेक्टरी निर्दिष्ट गर्न सक्नुहुन्छ।


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` वस्तुमा `encode` कार्य हुन्छ जसलाई पाठलाई सिधै एन्कोड गर्न प्रयोग गर्न सकिन्छ:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

त्यसपछि, प्रशिक्षणको क्रममा डाटा पहुँच गर्न प्रयोग गरिने इटरेटरहरू सिर्जना गरौं। किनभने BERT ले आफ्नो नै कोडिङ्ग फङ्सन प्रयोग गर्दछ, हामीले पहिले परिभाषित गरेको `padify` जस्तै एक padding फङ्सन परिभाषित गर्न आवश्यक हुनेछ:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

हाम्रो अवस्थामा, हामी `bert-base-uncased` नामको प्रि-ट्रेन गरिएको BERT मोडेल प्रयोग गर्नेछौं। `BertForSequenceClassfication` प्याकेज प्रयोग गरेर मोडेल लोड गरौं। यसले सुनिश्चित गर्दछ कि हाम्रो मोडेलसँग पहिले नै वर्गीकरणको लागि आवश्यक आर्किटेक्चर छ, अन्तिम क्लासिफायर सहित। तपाईंले चेतावनी सन्देश देख्नुहुनेछ जसले अन्तिम क्लासिफायरको वजनहरू आरम्भ गरिएको छैन भनेर जनाउँछ, र मोडेललाई प्रि-ट्रेनिङ आवश्यक पर्नेछ - यो पूर्ण रूपमा ठीक छ, किनभने हामीले गर्न लागेको कुरा यही हो!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

अब हामी प्रशिक्षण सुरु गर्न तयार छौं! किनभने BERT पहिले नै प्रि-ट्रेन गरिएको छ, हामी सुरुवातमा सानो learning rate प्रयोग गर्न चाहन्छौं ताकि प्रारम्भिक weights बिग्रन नजाओस्।

सबै कठिन काम `BertForSequenceClassification` मोडेलले गर्छ। जब हामी प्रशिक्षण डाटामा मोडेललाई कल गर्छौं, यसले इनपुट मिनिब्याचको लागि loss र नेटवर्क आउटपुट दुवै फिर्ता गर्छ। हामी loss लाई parameter optimization को लागि प्रयोग गर्छौं (`loss.backward()` ले backward pass गर्छ), र `out` लाई प्रशिक्षण accuracy गणना गर्न प्रयोग गर्छौं, जुन `argmax` प्रयोग गरेर प्राप्त गरिएको labels `labs` लाई अपेक्षित `labels` सँग तुलना गरेर गणना गरिन्छ।

प्रक्रियालाई नियन्त्रण गर्न, हामी loss र accuracy लाई केही iterations सम्म जम्मा गर्छौं, र प्रत्येक `report_freq` प्रशिक्षण चक्रहरूमा तिनीहरूलाई प्रिन्ट गर्छौं।

यो प्रशिक्षण सम्भवतः धेरै समय लाग्न सक्छ, त्यसैले हामी iterations को संख्या सीमित गर्छौं।


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


तपाईं देख्न सक्नुहुन्छ (विशेष गरी यदि तपाईं पुनरावृत्तिहरूको संख्या बढाउनुहुन्छ र पर्याप्त समय पर्खनुहुन्छ) कि BERT वर्गीकरणले हामीलाई धेरै राम्रो शुद्धता दिन्छ! यसको कारण BERT ले भाषा संरचनालाई पहिले नै राम्रोसँग बुझिसकेको छ, र हामीले अन्तिम वर्गीकरणलाई मात्र सुधार गर्न आवश्यक छ। तर, किनभने BERT ठूलो मोडेल हो, सम्पूर्ण प्रशिक्षण प्रक्रिया लामो समय लिन्छ, र गम्भीर कम्प्युटेशनल शक्ति आवश्यक पर्छ! (GPU, र सम्भवतः एकभन्दा बढी GPU).

> **Note:** हाम्रो उदाहरणमा, हामीले सबैभन्दा सानो प्रि-ट्रेन गरिएको BERT मोडेल प्रयोग गरिरहेका छौं। ठूला मोडेलहरू छन् जसले सम्भवतः अझ राम्रो नतिजा दिन सक्छ।


## मोडेलको प्रदर्शन मूल्याङ्कन गर्दै

अब हामी हाम्रो मोडेलको प्रदर्शन परीक्षण डाटासेटमा मूल्याङ्कन गर्न सक्छौं। मूल्याङ्कन लूप प्रशिक्षण लूपसँग धेरै मिल्दोजुल्दो छ, तर हामीले `model.eval()` कल गरेर मोडेललाई मूल्याङ्कन मोडमा स्विच गर्न बिर्सनु हुँदैन।


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## मुख्य कुरा

यस युनिटमा, हामीले **transformers** पुस्तकालयबाट प्रि-ट्रेन गरिएको भाषा मोडेल लिनु र यसलाई हाम्रो पाठ वर्गीकरण कार्यमा अनुकूलन गर्नु कति सजिलो छ भन्ने देख्यौं। यसैगरी, BERT मोडेलहरूलाई इकाई निकासी, प्रश्न उत्तर, र अन्य NLP कार्यहरूको लागि प्रयोग गर्न सकिन्छ।

Transformer मोडेलहरू NLP मा हालको अत्याधुनिक प्रविधिको प्रतिनिधित्व गर्छन्, र प्रायः अवस्थामा, जब तपाईं अनुकूल NLP समाधानहरू कार्यान्वयन गर्न थाल्नुहुन्छ, यो पहिलो समाधान हुनुपर्छ जससँग तपाईंले प्रयोग गर्न सुरु गर्नुहुन्छ। तर, यदि तपाईं उन्नत न्यूरल मोडेलहरू निर्माण गर्न चाहनुहुन्छ भने, यस मोड्युलमा छलफल गरिएका पुनरावर्ती न्यूरल नेटवर्कहरूको आधारभूत सिद्धान्तहरू बुझ्नु अत्यन्त महत्त्वपूर्ण छ।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धि हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
